In [1]:
pip install firebase-admin

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
    --------------------------------------- 0.3/12.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.7 MB 1.5 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/12.7 MB 1.4 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/12.7 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.6/12.7 MB 1.5 MB/s eta 0:00:08
   --

In [7]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install flask-sock

Note: you may need to restart the kernel to use updated packages.


In [9]:
import firebase_admin
from firebase_admin import credentials, db  # Realtime Database 관련 모듈

from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from flask_sock import Sock 
from datetime import datetime

import json
# 앱 초기화
app = Flask(__name__)
sock = Sock(app)
CORS(app,
     resources = {r"/sensor/*": {"origins": ["http://localhost:3000"]}},
     supports_credentials=True
     )

# Firebase 초기화
if not firebase_admin._apps:
    cred = credentials.Certificate("./key/serviceKey.json")  # Firebase 서비스 계정 키 파일의 경로
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://bareu-2fa47-default-rtdb.firebaseio.com/'  
    })


sensor_set = {"status": "OFF"}
clients = []

# # Firebase 데이터경로
# db_ref = db.reference('sensor_data') # 데이터 저장 및 읽기 경로

@app.get("/")
def index():
     return render_template('index.html')
 
@app.get("/send-data")
def to_input():
    return render_template('testInput.html')

@app.post("/send-data")
def send_firebase():

    data = request.form.to_dict()
    custom_key = f"test_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"
    print(custom_key)
    db_ref = db.reference("test")

    db_ref.child(custom_key).set({"value": data["test"]})

    return jsonify({
        "message": "send data success",
        "firebase_key": custom_key,
        "data": data
    }),200


def sensor_status_update(data):
    for client in clients:
        try:
            client.send(data)
        except:
            clients.remove(client)

@app.get("/sensor/on")
def sensor_on() :
    print("센서연결")
    sensor_set["status"] = "ON"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"센서연결 완료!","sensorSet": True}),200

@app.get("/sensor/pause")
def sensor_pause() :
    print("센서 기록정지")
    sensor_set["status"] = "OFF"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"기록정지 완료!","sensorSet": False}),200

@app.get("/sensor/off")
def sensor_off() :
    print("센서 종료")
    sensor_set["status"] = "OFF"
    print(sensor_set["status"])
    sensor_status_update(sensor_set["status"])
    print(f"Value updated to: {sensor_set}")
    return jsonify({"message":"중지 완료!","sensorSet": False}),200

@sock.route('/ws')
def websocket(ws):
    print(f"Client Connected! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    clients.append(ws)
    try:
        while True:
            message = ws.receive()
            print("Received from ESP32:", message)
            # ESP32가 전송한 데이터를 출력
            for client in clients:
                client.send("Data received!")
    except Exception as e:
        print(f"Client Disconnected: {e}")
        clients.remove(ws)

    

if __name__ =="__main__":
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader = False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.48:5000
Press CTRL+C to quit
192.168.219.48 - - [17/Dec/2024 09:24:54] "GET /sensor/on HTTP/1.1" 200 -


센서연결
ON
Value updated to: {'status': 'ON'}


192.168.219.48 - - [17/Dec/2024 09:25:02] "GET /sensor/pause HTTP/1.1" 200 -


센서 기록정지
OFF
Value updated to: {'status': 'OFF'}


192.168.219.48 - - [17/Dec/2024 09:26:04] "GET /sensor/on HTTP/1.1" 200 -


센서연결
ON
Value updated to: {'status': 'ON'}


192.168.219.48 - - [17/Dec/2024 09:26:07] "GET /sensor/pause HTTP/1.1" 200 -


센서 기록정지
OFF
Value updated to: {'status': 'OFF'}
